<a href="https://colab.research.google.com/github/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decoding_109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid", palette="pastel", color_codes=True)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import os
import csv

In [2]:
##Download data file, set name

!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/109/109_Data_Final.csv?raw=true

!mv 136_Data_Final.csv\?raw\=true 136_Data_Final.csv
!mv 109_Data_Final.csv\?raw\=true 109_Data_Final.csv

--2019-04-17 08:10:25--  https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
Resolving github.com (github.com)... 192.30.255.113, 192.30.255.112
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-17 08:10:25--  https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-17 08:10:25--  https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv
Resolving raw.githubus

In [3]:
#Import data to dataframe
##Implement Random forest stuff here...
df = pd.read_csv('136_Data_Final.csv')
df.set_index('Subject')
df.head()

df_109 = pd.read_csv('109_Data_Final.csv')
df_109.set_index('Subject')

,Unnamed: 0,Block,Trial,Time,O2,O1,OZ,PZ,P4,CP4,...,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
Subject,,,,,,,,,,,,,,,,,,,,,
109,0,1,1,-200,-2.4540,-7.4055,-3.7317,1.9958,-0.3961,6.4609,...,9.8221,12.2913,16.4498,5.1061,14.7567,39.3748,14.5422,10.6530,7.2805,7.0
109,1,1,1,-196,-3.7176,-10.9205,-5.6706,3.7010,-0.5067,5.8601,...,8.0161,10.2671,14.9112,5.1602,14.8205,29.3318,14.7717,17.7262,5.2050,7.0
109,2,1,1,-192,-6.3099,-14.5290,-8.9660,4.2277,-1.1666,4.9355,...,4.7711,6.3217,11.6578,1.8895,11.7379,31.0242,6.2631,17.7477,1.9524,7.0
109,3,1,1,-188,-7.3233,-13.6481,-9.2608,5.3117,-0.7020,4.9301,...,2.0940,2.9055,9.4231,-0.8641,8.4070,17.0047,9.1701,15.7770,-6.2897,7.0
109,4,1,1,-184,-6.6049,-10.7886,-7.4505,6.4961,-0.3101,4.8077,...,0.7413,1.2050,9.1359,-2.1851,7.0578,31.5447,28.4067,13.9127,13.4890,7.0
109,5,1,1,-180,-5.8761,-9.4650,-6.4137,6.0440,-1.6662,3.2746,...,0.0737,0.5549,8.2314,-2.8626,6.6793,20.5486,5.9248,11.7926,-9.2164,7.0
109,6,1,1,-176,-4.9485,-7.1693,-5.2465,5.2205,-2.5981,2.5221,...,0.3043,0.7138,6.1935,-2.9951,5.9663,1.3211,-10.3782,7.0524,-14.9233,7.0
109,7,1,1,-172,-3.3862,-3.5359,-3.1586,4.7090,-2.4668,2.7920,...,1.0243,1.0963,4.9345,-2.2770,4.9263,7.2703,-4.4346,14.6373,-3.4949,7.0
109,8,1,1,-168,-2.1085,-1.6439,-1.4426,3.5299,-2.6505,2.2707,...,0.5447,0.3008,4.0784,-1.1520,3.9824,4.7820,-0.3519,11.8092,-7.8560,7.0


In [0]:
#df_one_hot_encoded = pd.get_dummies(df)
#df = df_one_hot_encoded


In [7]:
df_1 = df.groupby(np.arange(len(df))//5).mean()
df_1['High Value'] = 0
df_1.loc[df_1['Value'] >5, 'High Value'] = 1
df_1.head(60)

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value,High Value
0,2,136,1,1,-192,-3.39620,-4.56512,-3.06214,-8.42260,-7.79246,...,-5.43550,-0.76010,-3.13550,-3.66620,-207.28608,56.66922,20.32440,-66.61436,1.0,0
1,7,136,1,1,-172,-5.20628,-0.32864,-2.77046,-9.07170,-12.00468,...,-2.41326,-1.93568,-2.28162,-3.04326,-106.94820,1.75514,25.80194,-56.04718,1.0,0
2,12,136,1,1,-152,-2.84608,5.15314,0.78680,-5.60850,-7.75482,...,-4.15404,-3.17186,-2.11862,-4.03626,68.62368,-68.33450,20.30014,-18.55150,1.0,0
3,17,136,1,1,-132,8.96336,4.41986,6.62588,5.94982,9.95042,...,4.52602,7.27456,1.10208,1.26568,165.87524,-90.30248,18.97206,16.57666,1.0,0
4,22,136,1,1,-112,4.36154,-2.21088,1.13674,5.53136,6.81236,...,5.28440,6.59148,4.29536,8.84988,182.50180,-86.96050,9.76776,32.49734,1.0,0
5,27,136,1,1,-92,1.79962,4.60384,3.35576,6.10372,1.47634,...,-0.28368,-1.62946,2.39350,0.10214,131.59946,-47.10480,-12.50178,38.19382,1.0,0
6,32,136,1,1,-72,7.31702,3.71566,5.81652,7.83620,10.13494,...,-6.38296,-5.15328,-0.88880,-0.77766,45.96038,7.06100,-16.65158,29.75986,1.0,0
7,37,136,1,1,-52,3.54224,-4.16302,-0.50312,6.08560,10.01846,...,-2.77510,-2.73240,-1.33546,0.35156,-42.48102,52.07448,-17.35840,15.17392,1.0,0
8,42,136,1,1,-32,-4.62888,-0.31314,-3.19088,-1.04696,-2.00892,...,5.18670,-0.27704,1.63624,1.21138,-102.86696,75.15104,-26.32326,6.31374,1.0,0
9,47,136,1,1,-12,-9.90632,-6.31166,-8.19508,-7.35694,-8.83172,...,6.44744,1.79376,0.33278,-0.25722,-134.97826,99.99146,-22.33126,2.69774,1.0,0


In [8]:
import random
random_samp_list = []
index_list = []
for i in range(1,201):
  index_list.append(i)


train_samp_list = random.sample(index_list,k=100)
  
test_list = []
test_list = [x for x in index_list if x not in train_samp_list]


#df_train = df['Trial'] == i
print(train_samp_list)
print(test_list)

[48, 121, 192, 127, 88, 149, 41, 102, 144, 33, 108, 153, 87, 49, 194, 181, 54, 11, 136, 86, 114, 40, 169, 193, 106, 46, 53, 90, 68, 79, 131, 24, 105, 120, 70, 85, 159, 72, 139, 37, 107, 183, 141, 147, 91, 94, 77, 32, 20, 23, 12, 124, 150, 173, 118, 143, 134, 73, 26, 187, 67, 172, 189, 57, 113, 179, 155, 199, 3, 21, 81, 154, 128, 200, 10, 64, 27, 180, 59, 18, 47, 190, 38, 188, 132, 109, 19, 51, 52, 43, 14, 58, 112, 130, 126, 84, 137, 122, 69, 28]
[1, 2, 4, 5, 6, 7, 8, 9, 13, 15, 16, 17, 22, 25, 29, 30, 31, 34, 35, 36, 39, 42, 44, 45, 50, 55, 56, 60, 61, 62, 63, 65, 66, 71, 74, 75, 76, 78, 80, 82, 83, 89, 92, 93, 95, 96, 97, 98, 99, 100, 101, 103, 104, 110, 111, 115, 116, 117, 119, 123, 125, 129, 133, 135, 138, 140, 142, 145, 146, 148, 151, 152, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 174, 175, 176, 177, 178, 182, 184, 185, 186, 191, 195, 196, 197, 198]


In [0]:
rfc = RandomForestClassifier(n_estimators=500, max_features='auto')
#rfc.fit(x_pd, y_pd);

In [10]:
df_1 = df_1.drop(['Unnamed: 0','Subject'], axis=1)
#df_2.head()

NameError: ignored

In [1]:
from sklearn.model_selection import GroupShuffleSplit
n_splits = 20
cv = GroupShuffleSplit(n_splits=n_splits, test_size=0.2)
#df_2 = df_1[(df_1['Value']==1) | (df_1['Value']==9)]
#df_2 = df_2.drop(['O1','O2','OZ','FP1','FP2','F3','FZ','F4','P8','TP8','P7','FC4','FT8','TP7','FCZ','F8','F7','FT7','VEOU','VEOL','HEOR','HEOR'], axis=1)
df_2 = df_1.drop(['VEOU','VEOL','HEOR','HEOL'], axis=1)
#df_2 = df_2[df_2['Time'] < 800]
#df_2 = df_2[df_2['Time'] > 400]
r_in = np.zeros(n_splits)
r_out = np.zeros(n_splits)
for i,(train, test) in enumerate(cv.split(df_2, groups=df_2['Trial'])):
  #train = df_2.index[train]
  #test = df_2.index[test]
  y = df_2['High Value']
  X = df_2.drop('High Value', axis=1).astype('float')
  rfc.fit(X.iloc[train].values, y.iloc[train])
  predicted = rfc.predict(X.iloc[train])
  actual = y.iloc[train]
  r_in[i] = np.corrcoef(predicted, actual)[0, 1]
  t_predicted = rfc.predict(X.iloc[test])
  t_actual = y.iloc[test]
  r_out[i] = np.corrcoef(t_predicted, t_actual)[0, 1]
  print(r_in[i], r_out[i])
print("Mean R_out for entire time series = %.3g +/- %.3g" % (r_out.mean(), r_out.std()))


NameError: ignored

In [11]:
df_109 = df.groupby(np.arange(len(df_109))//5).mean()
df_109['High Value'] = 0
df_109.loc[df_109['Value'] >5, 'High Value'] = 1
df_109.head(60)


ValueError: ignored

In [0]:
plt.scatter(t_predicted,predicted)

In [0]:
df_train = df.drop(df.index[train_samp_list])
df_test = df.drop(df.index[test_list])

    
df_x = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)
df_y = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)

In [0]:
y_pd = df_y['Value']
x_pd = df_x

In [0]:
y_pd_predicted = rfc.predict(x_pd)
#formula
#def report(y, y_predicted):
 # conf_mat = confusion_marix(y, y_predicted)
  #report = pd.DataFrame(index=['Value'])
  
print(y_pd_predicted)

[1. 1. 1. ... 3. 3. 3.]
